In [1]:
import cv2
import dlib
import numpy as np

# Load the pre-trained face detector from dlib
face_detector = dlib.get_frontal_face_detector()

# Load the pre-trained facial landmarks predictor from dlib
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Load the pre-trained gender classification model
gender_model = cv2.dnn.readNetFromCaffe('deploy_gender.prototxt', 'gender_net.caffemodel')

def detect_gender(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert the RGB image to grayscale
    gray = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)

    # Detect faces in the grayscale image
    faces = face_detector(gray)

    for face in faces:
        # Extract the region of interest (face) from the RGB image
        face_roi = rgb_image[face.top():face.bottom(), face.left():face.right()]

        # Preprocess the face ROI for gender classification
        face_blob = cv2.dnn.blobFromImage(face_roi, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)

        # Set the blob as input to the gender classification model
        gender_model.setInput(face_blob)

        # Perform forward pass to get the predicted gender
        gender_preds = gender_model.forward()

        # Get the gender label (either 'Male' or 'Female')
        gender_label = 'Male' if gender_preds[0][0] > gender_preds[0][1] else 'Female'

        # Draw bounding box and gender label on the image
        cv2.rectangle(image, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 0), 2)
        # add text to the image
        cv2.putText(image, gender_label, (face.left(), face.top() - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return image

# loop through all the images
for i in range(1, 3):
    # Call the gender detection function
    image = detect_gender(f'image{i}.jpg')
    # Display the output image
    cv2.imshow("Detect Image", image)
    # keep the window open until we press a key
    cv2.waitKey(0)
cv2.destroyAllWindows()